In [1]:
using DelimitedFiles
using Random
using Turing
using LinearAlgebra
using Distributions

data = readdlm("../../data/data.txt")
t, y, yerr = data[:, 1], data[:, 2], data[:, 3]

include("EnSS_code_opti.jl")

move (generic function with 1 method)

In [2]:

n_schools = 8
y = [28.0, 8.0, -3.0, 7.0, -1.0, 1.0, 18.0, 12.0] # estimated treatment effects
σ = [15.0, 10.0, 16.0, 11.0, 9.0, 11.0, 10.0, 18.0] # standard error of the effect estimate
# @model function schools(y, σ)
#     μ ~ Normal(0, 5)
#     τ ~ truncated(Cauchy(0, 5), lower = 0)
#     θ ~ filldist(Normal(μ, τ), n_schools)
#     for i in 1:n_schools
#         y[i] ~ Normal(θ[i], σ[i])
#     end
# end

@model function school_reparam(y, σ)
    μ ~ Normal(0, 5)
    τ ~ truncated(Cauchy(0, 5), lower = 0)
    θ ~ filldist(Normal(0, 1), n_schools)
    # y .~ ForEach(μ .+ τ .* θ, σ)
    for i in 1:n_schools
        y[i] ~ Normal(μ + τ * θ[i], σ[i])
    end
end
mymod = school_reparam(y, σ)

Model{typeof(school_reparam), (:y, :σ), (), (), Tuple{Vector{Float64}, Vector{Float64}}, Tuple{}, DynamicPPL.DefaultContext}(school_reparam, (y = [28.0, 8.0, -3.0, 7.0, -1.0, 1.0, 18.0, 12.0], σ = [15.0, 10.0, 16.0, 11.0, 9.0, 11.0, 10.0, 18.0]), NamedTuple(), DynamicPPL.DefaultContext())

In [3]:
# fun(x,α,β,γ) = @. α * x^2 + β * x + γ
# x = 0.0:0.5:15.0

# @model function InferenceModel(y, t)
# 	α ~ Normal(2, 1)
# 	β ~ Normal(0, 1)
# 	γ ~ Normal(-2, 1)
# 	return y ~ MvNormal(fun(t, α, β, γ), I)
# end

# mymod = InferenceModel(y, t)
spl = EnSS(50)

EnSS{(), EnsembleSliceSampler{Float64, Int64}}(EnsembleSliceSampler{Float64, Int64}(1.0, 50, 50, 10000))

In [4]:
rng = Random.MersenneTwister(0)
chain = sample(rng,mymod, spl, 20_000,progress=true);

Sampling   0%|                                          |  ETA: N/A
Sampling   0%|▎                                         |  ETA: 0:42:02
Sampling   1%|▍                                         |  ETA: 0:22:42
Sampling   2%|▋                                         |  ETA: 0:16:06
Sampling   2%|▉                                         |  ETA: 0:12:50
Sampling   2%|█                                         |  ETA: 0:11:02
Sampling   3%|█▎                                        |  ETA: 0:09:38
Sampling   4%|█▌                                        |  ETA: 0:08:42
Sampling   4%|█▋                                        |  ETA: 0:07:57
Sampling   4%|█▉                                        |  ETA: 0:07:21
Sampling   5%|██▏                                       |  ETA: 0:06:52
Sampling   6%|██▎                                       |  ETA: 0:06:29
Sampling   6%|██▌                                       |  ETA: 0:06:09
Sampling   6%|██▊                                       |  ETA: 0:05

InterruptException: InterruptException:

In [5]:
chain

UndefVarError: UndefVarError: `chain` not defined